# ウェブスクレイピング

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/shinchu/dataviz-notebooks/blob/main/week_5/web-scraping.ipynb)

ウェブスクレイピングは、ウェブページの内容を取得する技術です。

静的なウェブページに対しては、HTMLを取得し、HTMLの構造を解析（パース）することで内容を抽出することができます。

動的なウェブページに対しては（アクセスがあって初めて内容が生成されるようなもの）、ブラウザの動きをエミュレートするライブラリを使って内容を取得します。

短時間に大量のアクセスをしてしまうと、相手側のサーバーに負担がかかるため、関連する法律やサイトの利用規約、倫理的な問題に気をつけて行いましょう（例：3秒に1回アクセスする、などの設定にする）。

## 静的ページのスクレイピング

* [requests](https://requests.readthedocs.io/en/latest/): 指定したURLに対してリクエストを投げてレスポンスを取得するライブラリ
* [BeautifulSoup](https://www.crummy.com/software/BeautifulSoup/bs4/doc/): HTMLやXMLをパースするライブラリ

### ページを取得する

[DIPEx Japan](https://www.dipex-j.org/)の認知症患者と家族の語りを収集します。

In [ ]:
import time # 時間の処理
import re # 正規表現
import requests
from bs4 import BeautifulSoup

In [ ]:
index_page_url = "https://www.dipex-j.org/dementia/profid"

In [ ]:
# HTMLを取得する
html = requests.get(index_page_url)

In [ ]:
# 取得したHTMLを解析する
soup = BeautifulSoup(html.content, "html.parser")

In [ ]:
# 生のHTMLを確認する
soup

In [ ]:
# 目当ての要素を取り出す（それぞれの語り手ページへのリンク）
elem = soup.select("div[class='profid-link'] > ul > li")

In [ ]:
# リンクをリストに格納する
links = []
for e in elem:
    link = e.select("a")[0].attrs["href"]
    links.append(link)

In [ ]:
# リンクを取得できた
links

---

まとめて処理するための前準備

In [ ]:
# 1つ目のリンクにアクセスして様子を見る

In [ ]:
html = requests.get(links[0])

In [ ]:
soup = BeautifulSoup(html.content, "html.parser")

In [ ]:
soup

In [ ]:
# 体験談一覧を取得する
elem = soup.select("li[class='interviewItem']")

In [ ]:
elem

In [ ]:
# ひとつひとつの体験談へのリンクを取得する
episode_links = []
for e in elem:
    link = e.select("a")[0].attrs["href"]
    episode_links.append(link)

In [ ]:
# エピソードリンクを取得できた
episode_links

In [ ]:
# 1つ目の体験談にアクセスしてみる

In [ ]:
html = requests.get(episode_links[0])

In [ ]:
soup = BeautifulSoup(html.content, "html.parser")

In [ ]:
soup

In [ ]:
# 語りの内容のテキストを取得する（これが最終的な目的）
katari = soup.select("div[class='interview-text-box']")
katari = katari[0].get_text().strip().replace("\n", "")

In [ ]:
katari

In [ ]:
# URLを見ると、トピックがあらかじめ分類されているようなので、ついでに取得する
episode_links[0]

In [ ]:
# 正規表現を使う
topic = re.search(r'/topic/(.*)/[0-9]+', episode_links[0]).groups()[0]

In [ ]:
topic

In [ ]:
# 語り手番号も取得しておく
links[0]

In [ ]:
katarite = re.search(r'/profile/(.*).html', links[0]).groups()[0]

In [ ]:
katarite

これで1人目の語り手の1つ目のエピソードを取得できたので、ループを回してこれを全体に適用する。

---

In [ ]:
# 保存用のファイルを作る
with open("./episodes.tsv", "w") as f:
    f.write("topic\tkatarite\tepisode\n")

In [ ]:
for link in links:
    html = requests.get(link)
    soup = BeautifulSoup(html.content, "html.parser")
    # URLから語り手番号を取得する
    katarite = re.search(r'/profile/(.*).html', link).groups()[0]
    # 体験談一覧を取得する
    elem = soup.select("li[class='interviewItem']")
    # ひとつひとつの体験談へのリンクを取得する
    episode_links = []
    for e in elem:
        link = e.select("a")[0].attrs["href"]
        episode_links.append(link)
    # 体験談にアクセスする
    for episode_link in episode_links:
        html = requests.get(episode_link)
        soup = BeautifulSoup(html.content, "html.parser")
        # 語りの内容のテキストを取得する
        katari = soup.select("div[class='interview-text-box']")
        katari = katari[0].get_text().strip().replace("\n", "")
        # URLからトピックを取得する
        topic = re.search(r'/topic/(.*)/[0-9]+', episode_link).groups()[0]
        # ファイルに保存する
        with open("./episodes.tsv", "a") as f:
            f.write(f"{topic}\t{katarite}\t{katari}\n")
        # 3秒休んでから次のページにアクセスする
        time.sleep(3)
    # 3秒休んでから次のページにアクセスする
    time.sleep(3)

## 動的ページのスクレイピング

* [Selenium](https://www.selenium.dev/ja/documentation/): ブラウザのエミュレーションをするライブラリ

In [ ]:
# Seleniumのインストール
!pip install selenium

In [ ]:
# WebDriverのインストール
!apt-get update
!apt install chromium-chromedriver
!cp /usr/lib/chromium-browser/chromedriver /usr/bin

In [ ]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC

In [ ]:
# WebDriverの設定
options = webdriver.ChromeOptions()
options.add_argument("--headless")
options.add_argument("--no-sandbox")
options.add_argument("--disable-dev-shm-usage")

In [ ]:
# WebDriverを呼び出す
driver = webdriver.Chrome("chromedriver", options=options)

In [ ]:
url = "https://google.com"

In [ ]:
# Webサイトにアクセスする
driver.get(url)

In [ ]:
# ページ内のすべての要素が読み込まれるまで待機する。ただし、10秒たったらタイムアウトする。
element = WebDriverWait(driver, 10).until(
    EC.presence_of_all_elements_located
)

In [ ]:
# 取得したHTMLを解析して、検索ボックスを選択する
search = driver.find_element(By.NAME, "q")

In [ ]:
# 検索キーワードを入力して、実行する
search.send_keys("data visualization")
search.submit()

element = WebDriverWait(driver, 10).until(
    EC.presence_of_all_elements_located
)

In [ ]:
# 検索結果を取得
result = driver.find_element(By.ID, "search")

In [ ]:
# 検索結果ののタイトルを表示
links = result.find_elements(By.TAG_NAME, "h3")
for link in links:
    print(link.text)